# Finantial Times Scraping

### Inicializações

In [4]:
import pandas as pd
from datetime import datetime, timedelta

### Scrapping Exchange Rates from www.exchangerates.org.uk

In [5]:
url = 'http://www.exchangerates.org.uk/USD-EUR-exchange-rate-history.html'
csv = 'exchange_rates.csv'
#csv = 'D:\Personal\Python\Fundos\exchange_rates.csv'

df = pd.read_html(url, skiprows=[0,1,182])[0][[0,1]].rename(columns={0:'Date',1:'USD2EUR'})
df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x, infer_datetime_format = True).date())
df['USD2EUR'] = df['USD2EUR'].apply(lambda x: float(x[8:15]))
df.set_index('Date',inplace=True)

df_historic_usd2eur = pd.read_csv(csv,sep=';',parse_dates=True)
df_historic_usd2eur['Date'] = df_historic_usd2eur['Date'].apply(lambda x: pd.to_datetime(x, infer_datetime_format = True).date())
df_historic_usd2eur.set_index('Date',inplace=True)

df_historic_usd2eur = pd.concat([df.loc[:max(df_historic_usd2eur.index)][:-1],df_historic_usd2eur])
df_historic_usd2eur.to_csv(csv,sep=';')

### Update Portofolio From Transactions

In [6]:
#transacoes_csv = 'D:\\Personal\\Python\\Fundos\\transacoes.csv'
transacoes_csv = 'transacoes.csv'

encoding='latin_1'
thousands = ','
decimal = '.'
to_date = lambda d: datetime.strptime(d, '%d-%m-%Y')
converters={'Data de subscrição': to_date}

portfolio_csv = 'portofolio.csv'
df_portofolio = pd.read_csv(transacoes_csv ,sep=',',encoding=encoding,thousands=thousands, decimal=decimal, converters=converters)
df_portofolio = df_portofolio[pd.isnull(df_portofolio['Data de resgate'])].groupby(['Code','Nome','Moeda'])['Quantidade'].sum().reset_index()
df_portofolio.to_csv(portfolio_csv,sep=';',index=False)
df_portofolio.sort_values(by='Nome')

,Code,Nome,Moeda,Quantidade
4,LU0171309270,BGF Healthscience E2 EUR,EUR,76.8300
5,LU0171310955,BGF Wor Techn EUR E,EUR,30.2400
8,LU0355585513,JPF Africa Equity D,EUR,14.7020
0,IE00BLP5S353,Jupiter Glb Abs Rt,USD,15864.5900
1,LU0094557526,MFS European Research A1 EUR,EUR,23.6460
2,LU0121204431,NN Glb Sustain Eq,EUR,9.0650
9,PTSQUGHM0007,Property Core Real Estate,EUR,1901.7213
7,LU0248173006,SIC Asia B accum EUR,EUR,105.4200
6,LU0244354824,SIC China opp B USD,USD,7.9000
3,LU0122613903,T Eastern Europe,EUR,32.1960


### Update with any downloaded data
### Update wiht scrapping Quotes from FT 

In [7]:
url = 'https://markets.ft.com/data/funds/tearsheet/historical?s={}:{}'
xls = 'historico_cotacoes.xlsx'
to_date = lambda d: datetime.strptime(d, '%Y-%m-%d')
converters={'date': to_date}

novo_dict_df = dict()

dict_df = {key.strip():value for key,value in pd.read_excel(xls,sheet_name=None).items()}

for symbol, df in dict_df.items():
    if symbol in df_portofolio['Code'].tolist():
        dict_df[symbol]['Date'] = dict_df[symbol]['Date'].apply(lambda x: x.date())
        novo_dict_df[symbol] = dict_df[symbol].set_index('Date',drop=True).sort_index(ascending=False)
    

for symbol, _, currency, _ in df_portofolio.itertuples(index=False):

    # try to get data from a csv dwoloaded from Morningstar
    try:
        path = '.\\cotacoes_morningstar\\{}.csv'.format(symbol)
        df = pd.read_csv(path ,sep=';',encoding=encoding,thousands=thousands, decimal=decimal, converters=converters)[['date','price']]
        df['date'] = df['date'].apply(lambda x: x.date())
        df = df.set_index('date',drop=True).sort_index(ascending=False).rename(columns={'price':'Close'})
        df.index.rename('Date',inplace=True)
        if symbol in dict_df.keys():
            novo_dict_df[symbol] = pd.concat([novo_dict_df[symbol], df.loc[df.index.difference(novo_dict_df[symbol].index)]]).sort_index(ascending=False)
        else:
            novo_dict_df[symbol] = df
    except:
        print("Warning: No "+path)

    # try to data from FT
    try:
        url = 'https://markets.ft.com/data/funds/tearsheet/historical?s={}:{}'.format(symbol,currency)
        df = pd.read_html(url)[0]
        df['Date'] = df['Date'].apply(lambda x: x[:-17]).apply(lambda x: pd.to_datetime(x, infer_datetime_format = True).date())
        df = df.set_index('Date',drop=True)[['Close']]
        novo_dict_df[symbol] = pd.concat([novo_dict_df[symbol], df.loc[df.index.difference(novo_dict_df[symbol].index)]]).sort_index(ascending=False)
    except:
        print("Warning: Invalid "+url)
        

In [8]:
with pd.ExcelWriter(xls, engine="openpyxl") as writer:
    for symbol, df in novo_dict_df.items():
        df.to_excel(writer, sheet_name=symbol)
